Realizar um modelo de DNN (Deep neural network) não é tão simples. Alguns problemas que podemos ter:
- Se deparar com os gradientes de fuga ou explosão de gradientes, que ocorre quando os gradientes ficam cada vez menores ou maiores quando circulam de maneira reversa pela DNN durante o treinamento. Isso dificulta o treinamento das camadas inferiores.
- Possibilidade de não ter dados de treinamento suficiente para uma rede tão grande ou pode custar os olhos da cara para rotular esses dados.
- Treinamento pode ser extremamente demorado
- Modelos com milhões de parâmetros correm o risco de overfitting no conjunto de treinamento, principalmente se não tiver instâncias de treinamento suficientes ou elas tiverem muito ruído

# Problemas de gradientes de fuga e explosão de gradientes

Quando o algoritmo de retropropagação passa da camada de saída para a camada de entrada, propagando o gradiente do erro ao longo do caminho, ele calcula o gradiente da função de custo em relação a cada parâmetro na rede e usa esses gradientes para atualizar cada parâmetro com uma etapa do GD.

Como os gradientes costumam ficar cada vez menores, a atualização do GD deixa os pesos de conexão das camadas inferiores praticamente inalterados e o treinamento nunca converge para uma boa solução. Isso se chama _gradiente de fuga_. Às vezes pode acontecer o inverso, onde os gradientes podem aumentar cada vez mais até as camadas receberem atualizações extremamente grandes de peso e o algoritmo divergir (Explosão de divergentes). Em geral, redes neurais profundas sofrem de gradientes instáveis.

# Inicialização de Glorot e inicialização de He

Para remediar os gradientes instáveis , Glorot e Bengio (link do artigo no livro) destacam que precisamos que o sinal flua corretamente nas duas direções: à frente (forward) ao fazer predições e na direção reversa (reverse) ao retropropagar os gradientes. Para que o sinal flua coretamente, os autores argumentam que precsamos que a variância das saídas de cada camada seja igual a variância de suas entradas, e precisamos que os gradientes tenham variância igual antes e depois de fluir através de uma camada na direção reversa.

A pincípio não é possível garantir nem um nem outro, a menos que a camada tenha um número igual de entradas (sendo chamados de _fan-in_ e _fan-out_ da camada). Os autores propuseram um bom meio-termo, os pesos de conexão de cada camada devem ser inicializados aleatoriamente:

### Definições

$$
fan_{avg} = \frac{fan_{in} + fan_{out}}{2}
$$

---

### Distribuição uniforme
Pesos 𝑤 pertencentes a uma distribuição uniforme no intervalo de menos raiz quadrada de três sobre fan médio até raiz quadrada de três sobre fan-avg.
$$
W \sim U\left(-\sqrt{\frac{3}{fan_{avg}}}, \; \sqrt{\frac{3}{fan_{avg}}}\right)
$$

---

### Distribuição normal
Pesos 𝑤 pertencentes a uma distribuição normal com média zero e variância igual a 1 sobre fan-avg.
$$
W \sim \mathcal{N}\left(0, \; \frac{1}{fan_{avg}}\right)
$$

Relação de funções de ativação e inicialização:
- Glorot: Para nenhuma função de ativação, ou tanh, logística, softmax
- He: ReLU e variantes
- LeCun: SELU

In [ ]:
#Esse código é apenas de demonstração e não será executado

# Por padrão, a Keras usa inicialização de Glorot com uma distribuição uniforme. Ao criar a camada, podemos mudar para a inicialização He
import tensorflow as tf
keras = tf.keras

keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal')

# Para uma inicialização com uma distribuição uniforme, mas com base em fan-avg em vez de fan-in, usar o inicializador VarianceScaling

he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fav_avg', distribution='uniform')
keras.layers.Dense(10, activation='sigmoid', kernel_initializer=he_avg_init)

# Funções de ativação de não saturação
Parte dos problemas com gradientes instáveis se dão pela má escolha da função de ativação. A função de ativação ReLU (que normalmente é utilizada em boa parte das redes neurais) não é perfeita, e sofre de _dying ReLUs_, ou seja, durante o treinamento, alguns neurônios "morrem" na prática, parando de gerar a saída de qualquer coisa que seja diferente de 0.

Para solucionar esse problema, convém usar uma variante da ReLU, como a _leaky ReLU_, que contém uma inclinação para $x < 0$, onde é definido pelo hiperparâmetro $\alpha$ 

$\text{LeakyReLU}(x) = \max(\alpha x, x)$

Temos alguns outros tipos de LeakyReLU, como o RReLU que randomiza o $\alpha$ em um determinado intervalo durante o treinamento e é fixado em um valor médio durante o teste. PReLU (Parametric ReLU), em que $\alpha$ é autorizado a ser aprendido durante o treinamento, se tornando um parâmetro que pode ser modificado durante a retropropagação, o que é muito útil em conjuntos de imagens grandes, mas que pode ocorrer overfitting em no conjunto de treinamento em datasets pequenos.

A função de ativação ELU (Exponential linear unity) ultrapassou em desempenho todas as variantes da ReLU, ela se parece bastante com a ReLU, mas com algumas diferenças:
- Ela assume valores negativos (ReLU transforma o gradiente da função em 0 quando a entrada é negativa) quando $x < 0$, possibilitando ter uma saída média próxima de 0 e ajuda a remediar o problema dos gradientes em fuga. O hiperparâmetro $\alpha$ define o valor que a função ELU se aproxima, quando $x$ é um número negativo grande. Geralmente é definido como 1, mas pode ser alterado como qualquer hiperparâmetro.
- Tem um gradiente diferente de 0 para $x < 0$, o que evita o problema dos neurônios mortos.
- Se $\alpha$ é igual a 1, a função é suavizada em todos os lugares, incluindo $x = 0$, coisa que ajuda a acelerar o gradiente descendente pois não oscila tanto para a esquerda e direita de $x = 0$

***
Equação ELU:

$$
\text{ELU}(x) =
\begin{cases}
x & \text{se } x \ge 0 \\
\alpha (e^x - 1) & \text{se } x < 0
\end{cases}
$$


O principal problema da função de ativação ELU é que ela é mais lenta que a função ReLU e variantes, devido ao uso da função exponencial. Sua taxa de convergência é mais rápida durante o treinamento, mas ainda assim no teste, uma rede ELU será mais lenta que uma rede ReLU.
***

Scaled ELU (SELU) é uma variante escalonada da ELU, onde os autores demonstraram que construir uma rede neural composta exclusivamente de uma pilha de camadas densas e, se todas as camadas ocultas usarem a função de ativação SELU, a rede se normalizará automaticamente. A saída de cada camada tende a preservar uma média 0 e um desvio-padrão 1 durante o treinamento, o que resolve o problema de fuga e explosão.
Condições para que a autonormalização ocorra:
- Características devem estar padronizadas (StandardScaler)
- Os pesos de cada camada oculta devem ser inicializados com a inicialização normal de LeCun = kernel_initializer='lecun_normal'
- A arquitetura da rede **deve ser sequencial**. SELU em redes não sequenciais , como redes recorrentes ou com _skip connections_, a autonormalização não será garantida.
- O artigo apenas assegura a autonormalização se todas as camadas forem densas, mas alguns pesquisadores observaram que a função de ativação SELU também pode melhorar o desempenho em redes neurais convolucionais.

Então basicamente: SELU > ELU > leaky ReLU > ReLU > tanh > logística

In [ ]:
# Para utilizar a ativação leaky ReLU, é só criar uma camada LeakyReLU() e adicionar ao modelo após a camada à qual quer aplicar

model = keras.models.Sequential([
    # [...]
    keras.layers.Dense(10, kernel_initializer='he_normal'),
    keras.layers.LeakyReLU(alpha=.2)
    # [...]
])

# Para a PReLU, substitua o alpha por PReLU().

#SELU defina activation='selu' e kernel_initializer='lecun_normal' quando criar a camada
layer = keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal')

# Normalização em batch
Embora o uso da inicialização He junto com a ELU (ou variantes da ReLU) possa reduzir significativamente o risco de problemas, de gradiente de fuga/explosão de gradientes no início do treinamento, isso não garante que eles não voltem durante o treinamento.

A normalização em batch (BN) aborda esses problams, consistindo em adicionar uma operção no modelo logo antes ou depois da função de ativação de cada camada oculta. Essa operação centraliza o zero e normaliza cada entrada, em seguida escalona e modifica o resultando usando dois novos vetores de parâmetros por camada: um para o escalonamento e outro para o deslocamento.

### Equações

Dado um batch \( B = \{x_1, x_2, ..., x_m\} \):

$$
\mu_B = \frac{1}{m} \sum_{i=1}^{m} x_i
$$

$$
\sigma_B^2 = \frac{1}{m} \sum_{i=1}^{m} (x_i - \mu_B)^2
$$

$$
\hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}
$$

$$
y_i = \gamma \hat{x}_i + \beta
$$

---

### Explicação

- $x_i$: valor de entrada de um neurônio para o exemplo $i$  
- $m$: tamanho do batch  
- $\mu_B$: média do batch (centraliza os dados)  
- $\sigma_B^2$: variância do batch (mede dispersão)  
- $\epsilon$: constante pequena (evita divisão por zero)  
- $\hat{x}_i$: valor normalizado (média 0, variância 1)  
- $\gamma$: parâmetro treinável de escala  
- $\beta$: parâmetro treinável de deslocamento  
- $y_i$: saída final após normalização e ajuste  

Durante o treinamento, a BN padroniza suas entradas, em seguida as reescalona e as desloca. Mas na hora de testar, talvez tenhamos que fazer predições para as instâncias individuais, e não para os batches de instâncias. Nesse caso não teremos como calcular a média e desvio-padrão de cada entrada. Além do mais, ainda que tenhamos um batch de instâncias, ele pode ser muito pequeno, ou as instâncias podem não ser independentes e distribuídas de forma idêntica, logo, os cálculos computacionais em relação ao batch de instância são duvidosos. Uma solução seria aguardar até o fim do treinamento, rodar todo o conjunto de treinamento pela rede neural, e calcular a média e o desvio padrão de cada entrada da camada BN. Assim, essas médias de entrada "finais" e desvios-padrão podem ser utilizados em vez das médias de entreda do batch e desvios-padrão ao fazer as predições. Contudo, a maioria das implementações da normalização em batch estima essas estatísticas finais durante o treinamento, utilizando uma média móvel de entrada e dos desvios-padrão da camada.